In [1]:
import os
import re
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# ==============================================================================
# 1. FUNCIÓN DE CARGA Y LIMPIEZA BÁSICA
# ==============================================================================
def cargar_reporte(path_archivo):
    print(f"\n{'='*60}")
    print(f"📂 CARGANDO: {path_archivo}")
    print(f"{'='*60}")
    
    if not os.path.exists(path_archivo):
        print("❌ Error: El archivo no existe.")
        return None

    try:
        # A. Detectar Cabecera (Capital IQ suele poner basura en las primeras filas)
        # Leemos las primeras 20 filas sin formato
        df_temp = pd.read_excel(path_archivo, header=None, nrows=20)
        
        start_row = 0
        for i, row in df_temp.iterrows():
            row_str = str(row.values)
            # Buscamos la fila que tiene los Tickers
            if "Ticker" in row_str or "Exchange:Ticker" in row_str:
                start_row = i
                break
        
        print(f"   -> Cabecera detectada en la fila: {start_row + 1}")
        
        # B. Cargar DataFrame Real
        df = pd.read_excel(path_archivo, header=start_row)
        
        # C. Limpieza de Valores No Numéricos
        # Capital IQ usa guiones '-' o 'NM' para datos faltantes. Esto rompe las estadísticas.
        # Reemplazamos todo lo que no sea número por NaN
        
        # Identificar columnas de datos (excluyendo Ticker y Nombre)
        cols_datos = [c for c in df.columns if "Ticker" not in str(c) and "Name" not in str(c)]
        
        # Convertir a numérico (fuerza errores a NaN)
        for col in cols_datos:
            df[col] = pd.to_numeric(df[col], errors='coerce')
            
        print(f"   -> Datos cargados y limpiados (Guiones convertidos a NaN).")
        return df

    except Exception as e:
        print(f"❌ Error crítico leyendo el archivo: {e}")
        return None

# ==============================================================================
# 2. FUNCIÓN DE CHEQUEOS ESTADÍSTICOS
# ==============================================================================
def hacer_sanity_checks(df):
    if df is None: return

    # 1. HEAD
    print("\n--- 1. VISTA PREVIA (HEAD) ---")
    print(df.iloc[:, :5].head(3)) # Mostramos solo las primeras 5 columnas para que quepa
    print("... (más columnas) ...")

    # 2. DIMENSIONES
    print(f"\n--- 2. DIMENSIONES ---")
    print(f"Filas (Empresas): {df.shape[0]}")
    print(f"Columnas (Periodos): {df.shape[1]}")

    # 3. NULOS (Vital para saber la calidad de la data)
    total_celdas = df.size
    total_nulos = df.isna().sum().sum()
    porcentaje_nulos = (total_nulos / total_celdas) * 100
    
    print(f"\n--- 3. CALIDAD DE DATOS ---")
    print(f"Celdas vacías (NaNs): {total_nulos} ({porcentaje_nulos:.2f}%)")
    
    if porcentaje_nulos > 50:
        print("⚠️ ADVERTENCIA: Más del 50% de los datos están vacíos. Revisa el archivo.")

    # 4. ESTADÍSTICA DESCRIPTIVA (Promedios Globales)
    print("\n--- 4. ESTADÍSTICA RÁPIDA (Última Columna/Periodo) ---")
    # Tomamos la última columna (asumiendo que es el dato más reciente)
    ultima_col = df.columns[-1]
    print(f"Columna analizada: '{ultima_col}'")
    print(df[ultima_col].describe())

# ==============================================================================
# 3. FUNCIÓN DE GRAFICACIÓN (Trend Check)
# ==============================================================================
def graficar_tendencia_promedio(df, titulo="Tendencia Promedio"):
    if df is None: return

    # Seleccionamos solo columnas numéricas
    cols_num = df.select_dtypes(include=[np.number])
    
    # Calculamos el promedio de TODAS las empresas para cada fecha (columna)
    # Esto nos dice: "¿El EPS promedio del S&P500 subió o bajó en estos 10 años?"
    promedios = cols_num.mean()
    
    plt.figure(figsize=(10, 5))
    promedios.plot(kind='line', color='blue', marker='o')
    
    plt.title(f"Check Visual: {titulo} (Promedio de todas las empresas)")
    plt.ylabel("Valor Promedio")
    plt.xlabel("Periodos (Columnas)")
    plt.grid(True, alpha=0.3)
    plt.xticks(rotation=45)
    plt.tight_layout()
    plt.show()

# ==============================================================================
# 4. FUNCIÓN DE PROCESAMIENTO GENÉRICO
# ============================================================================= 
def procesar_reporte_generico(archivo_entrada, texto_busqueda, nombre_metrica_salida):
    """
    Limpia cualquier reporte trimestral de Capital IQ (Formato LTM CQ).
    
    Args:
        archivo_entrada (str): Nombre del archivo .xls
        texto_busqueda (str): Parte del nombre de la columna para identificarla (ej: "Interest Exp")
        nombre_metrica_salida (str): Nombre final para la base de datos (ej: "interest_coverage")
    """
    print(f"\n🚀 Procesando: {archivo_entrada}...")
    
    if not os.path.exists(archivo_entrada):
        print(f"   ❌ Error: No encuentro el archivo {archivo_entrada}")
        return None

    # 1. Cargar Excel (Buscando cabecera automáticamante)
    try:
        df_temp = pd.read_excel(archivo_entrada, header=None, nrows=15)
        start_row = 0
        for i, row in df_temp.iterrows():
            if "Ticker" in str(row.values) or "Exchange:Ticker" in str(row.values):
                start_row = i
                break
        
        df = pd.read_excel(archivo_entrada, header=start_row)
    except Exception as e:
        print(f"   ❌ Error leyendo Excel: {e}")
        return None

    print(f"   -> Columnas originales encontradas: {len(df.columns)}")

    # 2. Configurar Regex y Mapa de Fechas
    quarter_map = {'CQ1': '03-31', 'CQ2': '06-30', 'CQ3': '09-30', 'CQ4': '12-31'}
    regex = re.compile(r'(CQ[1-4])\s+(\d{4})') # Captura "CQ1 2000"
    
    new_columns = {}
    
    # 3. Renombrar Columnas Dinámicamente
    for col in df.columns:
        col_str = str(col)
        
        # Identificadores
        if "Exchange:Ticker" in col_str:
            new_columns[col] = "ticker"
        elif "Company Name" in col_str:
            new_columns[col] = "company_name"
        
        # Métrica Variable (Aquí usamos el texto de búsqueda)
        elif texto_busqueda in col_str:
            match = regex.search(col_str)
            if match:
                q = match.group(1)
                y = match.group(2)
                date_str = f"{y}-{quarter_map[q]}"
                new_columns[col] = date_str
    
    # Aplicar renombrado
    df = df.rename(columns=new_columns)
    
    # 4. Filtrar y Melt
    # Nos quedamos solo con Ticker, Nombre y las columnas que logramos convertir a Fecha
    cols_validas = ['ticker', 'company_name'] + [c for c in df.columns if re.match(r'\d{4}-\d{2}-\d{2}', str(c))]
    df = df[cols_validas]
    
    df_long = df.melt(
        id_vars=['ticker', 'company_name'], 
        var_name='date', 
        value_name='value'
    )
    
    # 5. Limpieza Final
    df_long['date'] = pd.to_datetime(df_long['date'])
    df_long['ticker'] = df_long['ticker'].apply(lambda x: str(x).split(':')[-1])
    df_long['value'] = pd.to_numeric(df_long['value'], errors='coerce')
    
    # Agregamos una columna con el nombre de la métrica (útil para SQL)
    df_long['metric_name'] = nombre_metrica_salida
    
    df_final = df_long.dropna(subset=['value']).sort_values(['ticker', 'date'])
    
    print(f"   ✅ ¡Listo! {len(df_final)} filas generadas.")
    return df_final


In [2]:


# Configuración Maestra de Reportes
config_reportes = [
    {
        "archivo": os.path.join("reportes", "1 - Total_Debt_EBITDA.xls"),
        "texto_clave": "Total Debt/EBITDA",
        "nombre_final": "total_debt/ebitda"
    },
    {
        "archivo": os.path.join("reportes", "2 - EBITDA_Interest_Coverage.xls"),
        "texto_clave": "Interest Exp",  # Busca "EBITDA / Interest Exp."
        "nombre_final": "EBITDA/interest_coverage"
    },
    {
        "archivo": os.path.join("reportes", "3 - Normalized_Diluted_EPS.xls"),
        "texto_clave": "Normalized Diluted EPS",
        "nombre_final": "normalized_diluted_EPS"
    },
    {
        "archivo": os.path.join("reportes", "4 - Levered_Free_Cash_Flow.xls"),
        "texto_clave": "Levered Free Cash Flow",
        "nombre_final": "levered_free_cash_flow"
    },
    {
        "archivo": os.path.join("reportes", "5 - Total_Revenue.xls"),
        "texto_clave": "Total Revenue",
        "nombre_final": "total_revenue"
    },
    {
        "archivo": os.path.join("reportes", "6 - EBIT.xls"),
        "texto_clave": "EBIT",
        "nombre_final": "ebit"
    },
    {
        "archivo": os.path.join("reportes", "7 -  Cash_Equivalents.xls"), # Ojo con los espacios del nombre original
        "texto_clave": "Cash and Equivalents",
        "nombre_final": "cash_equivalents"
    },
    {
        "archivo": os.path.join("reportes", "8  - Total_Equity .xls"), # Ojo con los espacios del nombre original
        "texto_clave": "Total Equity",
        "nombre_final": "total_equity"
    },
    {
        "archivo": os.path.join("reportes", "9 - Gross_Margin_%.xls"),
        "texto_clave": "Gross Margin",
        "nombre_final": "gross_margin"
    }
]

In [3]:
for reportes in config_reportes:

    # Probamos con uno solo para no llenar la pantalla, o haz un bucle
    archivo_prueba = reportes["archivo"] 

    # 1. Cargar
    df_raw = cargar_reporte(archivo_prueba)

    # 2. Chequear
    # hacer_sanity_checks(df_raw)

    # 3. Graficar
    # El título del gráfico será el nombre del archivo
    # graficar_tendencia_promedio(df_raw, titulo=archivo_prueba.replace(".xls", ""))
    print("colums:", df_raw.columns)


📂 CARGANDO: reportes/1 - Total_Debt_EBITDA.xls
   -> Cabecera detectada en la fila: 8
   -> Datos cargados y limpiados (Guiones convertidos a NaN).
colums: Index(['Company Name', 'Exchange:Ticker',
       'Index Constituents [Primary Listing]',
       'Index Constituents [Secondary Listings]',
       'Total Debt/EBITDA [LTM CQ1 1980]', 'Total Debt/EBITDA [LTM CQ2 1980]',
       'Total Debt/EBITDA [LTM CQ3 1980]', 'Total Debt/EBITDA [LTM CQ4 1980]',
       'Total Debt/EBITDA [LTM CQ1 1981]', 'Total Debt/EBITDA [LTM CQ2 1981]',
       ...
       'Total Debt/EBITDA [LTM CQ3 2023]', 'Total Debt/EBITDA [LTM CQ4 2023]',
       'Total Debt/EBITDA [LTM CQ1 2024]', 'Total Debt/EBITDA [LTM CQ2 2024]',
       'Total Debt/EBITDA [LTM CQ3 2024]', 'Total Debt/EBITDA [LTM CQ4 2024]',
       'Total Debt/EBITDA [LTM CQ1 2025]', 'Total Debt/EBITDA [LTM CQ2 2025]',
       'Total Debt/EBITDA [LTM CQ3 2025]', 'Total Debt/EBITDA [LTM CQ4 2025]'],
      dtype='object', length=188)

📂 CARGANDO: reportes/2 - 

In [5]:
for reportes in config_reportes:
    archivo = reportes["archivo"]
    texto_clave = reportes["texto_clave"]
    nombre_final = reportes["nombre_final"]

    df_clean = procesar_reporte_generico(archivo, texto_clave, nombre_final)

    if df_clean is not None:
        print(df_clean[df_clean['ticker'] == 'AAPL'].sort_values(by='date', ascending=True).head(2))
        print(df_clean[df_clean['ticker'] == 'AAPL'].sort_values(by='date', ascending=False).head(2))
        # Opcional: Guardar a CSV para verificar
        #df_clean.to_csv(f"/home/nromero/FinExp-/clean_reports/clean_{nombre_final}.csv")


🚀 Procesando: reportes/1 - Total_Debt_EBITDA.xls...
   -> Columnas originales encontradas: 188
   ✅ ¡Listo! 54233 filas generadas.
      ticker                company_name       date  value        metric_name
19111   AAPL  Apple Inc. (NasdaqGS:AAPL) 1989-09-30  0.075  total_debt/ebitda
21119   AAPL  Apple Inc. (NasdaqGS:AAPL) 1990-09-30  0.129  total_debt/ebitda
      ticker                company_name       date  value        metric_name
91399   AAPL  Apple Inc. (NasdaqGS:AAPL) 2025-09-30  0.690  total_debt/ebitda
90897   AAPL  Apple Inc. (NasdaqGS:AAPL) 2025-06-30  0.646  total_debt/ebitda

🚀 Procesando: reportes/2 - EBITDA_Interest_Coverage.xls...
   -> Columnas originales encontradas: 188
   ✅ ¡Listo! 44141 filas generadas.
      ticker                company_name       date  value  \
27143   AAPL  Apple Inc. (NasdaqGS:AAPL) 1993-09-30   4.21   
28147   AAPL  Apple Inc. (NasdaqGS:AAPL) 1994-03-31   9.13   

                    metric_name  
27143  EBITDA/interest_coverage  
28147 

In [6]:
import sqlite3
import pandas as pd
import os

# (Asegúrate de tener importada tu función procesar_reporte_generico aquí)
# from limpieza_maestra import procesar_reporte_generico, config_reportes

# ==============================================================================
# CREACIÓN DE LA BASE DE DATOS SQLITE
# ==============================================================================

def guardar_en_sqlite(config_reportes):
    nombre_db = "financial_data.db"
    
    print(f"🔌 Conectando a la base de datos: {nombre_db}...")
    conn = sqlite3.connect(nombre_db)
    
    # Iteramos sobre tu configuración
    for reportes in config_reportes:
        archivo = reportes["archivo"]
        texto_clave = reportes["texto_clave"]
        
        # Usamos nombre_final para que la tabla se llame 'total_debt' y no 'Total Debt/EBITDA'
        nombre_tabla = reportes["nombre_final"] 

        print(f"\n🏗️  Procesando: {nombre_tabla}...")

        # 1. Procesar el Excel
        df_clean = procesar_reporte_generico(archivo, texto_clave, nombre_tabla)

        if df_clean is not None:
            # Check rápido
            print(f"   -> Muestra (AAPL):")
            print(df_clean[df_clean['ticker'] == 'AAPL'].sort_values(by='date', ascending=False).head(2))
            
            # 2. Guardar en SQL
            # if_exists='replace': Borra la tabla si existe y la crea de nuevo (ideal para recargas)
            # index=False: No guardamos el índice numérico de pandas (0, 1, 2...)
            try:
                df_clean.to_sql(name=nombre_tabla, con=conn, if_exists='replace', index=False)
                print(f"   ✅ Tabla '{nombre_tabla}' guardada exitosamente ({len(df_clean)} filas).")
            except Exception as e:
                print(f"   ❌ Error guardando en SQL: {e}")
        else:
            print(f"   ⚠️ Saltando {nombre_tabla} (DataFrame vacío o error en lectura).")

    # Cerrar conexión
    conn.close()
    print(f"\n🎉 Proceso finalizado. Base de datos: {nombre_db}")

# ==============================================================================
# VERIFICACIÓN (Leer desde SQL para probar que funcionó)
# ==============================================================================
def verificar_db():
    conn = sqlite3.connect("financial_data.db")
    cursor = conn.cursor()
    
    # Listar todas las tablas creadas
    cursor.execute("SELECT name FROM sqlite_master WHERE type='table';")
    tablas = cursor.fetchall()
    
    print("\n📋 TABLAS EN LA BASE DE DATOS:")
    for t in tablas:
        print(f" - {t[0]}")
        
    conn.close()

if __name__ == "__main__":
    guardar_en_sqlite(config_reportes)
    verificar_db()

🔌 Conectando a la base de datos: financial_data.db...

🏗️  Procesando: total_debt/ebitda...

🚀 Procesando: reportes/1 - Total_Debt_EBITDA.xls...
   -> Columnas originales encontradas: 188
   ✅ ¡Listo! 54233 filas generadas.
   -> Muestra (AAPL):
      ticker                company_name       date  value        metric_name
91399   AAPL  Apple Inc. (NasdaqGS:AAPL) 2025-09-30  0.690  total_debt/ebitda
90897   AAPL  Apple Inc. (NasdaqGS:AAPL) 2025-06-30  0.646  total_debt/ebitda
   ✅ Tabla 'total_debt/ebitda' guardada exitosamente (54233 filas).

🏗️  Procesando: EBITDA/interest_coverage...

🚀 Procesando: reportes/2 - EBITDA_Interest_Coverage.xls...
   -> Columnas originales encontradas: 188
   ✅ ¡Listo! 44141 filas generadas.
   -> Muestra (AAPL):
      ticker                company_name       date  value  \
85375   AAPL  Apple Inc. (NasdaqGS:AAPL) 2022-09-30   38.6   
84873   AAPL  Apple Inc. (NasdaqGS:AAPL) 2022-06-30   36.0   

                    metric_name  
85375  EBITDA/interest_co

In [7]:
verificar_db()


📋 TABLAS EN LA BASE DE DATOS:
 - total_debt/ebitda
 - EBITDA/interest_coverage
 - normalized_diluted_EPS
 - levered_free_cash_flow
 - total_revenue
 - ebit
 - cash_equivalents
 - total_equity
 - gross_margin
